# XGBoost 

Noah Rubin

Self Study - July 2021

Main resources used:

* https://www.youtube.com/watch?v=ufHo8vbk6g4&t=0s
* https://xgboost.readthedocs.io/en/stable/tutorials/model.html
* https://maoweikai.top/xgboost.html
* https://stats.stackexchange.com/questions/202858/xgboost-loss-function-approximation-with-taylor-expansion



---

Intro:
* XGBoost (Extreme Gradient Boosting) is a scalable and highly accurate implementation of gradient boosting that pushes the limits of computing power for boosted tree algorithms, being built largely for energizing machine learning model performance and computational speed 
* With XGBoost, trees are built in parallel, instead of sequentially like GBDT
* It follows a level-wise strategy, scanning across gradient values and using these partial sums to evaluate the quality of splits at every possible split in the training set. 
* XGBoost is an algorithm that has recently been dominating applied machine learning and Kaggle competitions for structured or tabular data.



## Math Details:

Suppose we have a dataset $\mathcal{D} = \{(\boldsymbol{x}_i, y_i)\}_{i=1}^n$ with $n$ observations, $m$ features and a response variable $y$, such that $\boldsymbol{x}_i \in \mathbb{R}^m$ and $y_i \in \mathbb{R}.$ We can define the prediction for row $i$ made by XGBoost as $K$ additive functions, in which each tree built contributes to the output, such that:

$$\hat{y}_i = \psi(\boldsymbol{x}_i) = \sum_{i=1}^K f_{k}(\boldsymbol{x}_i)$$

Similarly, the prediction at the $t^{th}$ step can be defined as:

$$\hat{y}_{i}^{[t]} = \sum_{k=1}^{t} f_k(\boldsymbol{x}_i)$$

Typically for regression we would use a loss function such as mean squared error given as:

$$L = \frac{1}{n} \sum_{i=1}^n (y_i - \hat{y}_i)^2$$

With XGBoost, regularisation is a critical component of the model. A good regularisation term controls the complexity of the model which prevents overfitting. We define 

$$\Omega = \gamma{T} + \frac{1}{2}\lambda \sum_{j=1}^T {w}_j^2$$

where $T$ is the number of leaves, and ${w}_j^2$ is the score on the $j^{th}$ leaf. The $\gamma$ and $\lambda$ terms control the degree of the regularisation and are hyperparameters we can tune. In the image below we see that

$$\Omega = 3\gamma + \frac{1}{2}\lambda (2^2 + 0.1^2 + (-1)^2)$$

since our leaf weights were 2, 0.1 and -1 (moving from left to right)

<img src="xgboost_regularisation.png" alt="Drawing" style="width: 500px;"/>

If we combine the loss function and the regularisation term together we have:

$$\text{objective} = L + \Omega$$

whereby $L$ controls the predictive power and the $\Omega$ regularisation component controls simplicity.

---

When using XGBoost, we optimise the objective using gradient descent. Given $\text{objective}(y, \hat{y})$ to optimise, we can find $\frac{\partial \text{obj}(y, \hat{y})}{\partial \hat{y}}$

at each iteration of gradient descent. Then we improve $\hat{y}$ along the direction of the gradient to minimise the objective. Recall the general definition of $\text{objective} = L + \Omega$. For an iterative algorithm, we can redefine the objective function as:

$$\text{obj}^{[t]} = \sum_{i=1}^n L(y_i, \hat{y}_{i}^{[t]}) + \sum_{i=1}^t \Omega(f_i)$$

Since we can define $\hat{y}_{i}^{[t]} = \sum_{k=1}^t f_k({\boldsymbol x}_i) = \hat{y}_{i}^{[t-1]} + f_t({\boldsymbol x}_i)$, with our tree prediction in the $t$ iteration (ie the $t^{th}$ tree output) given as $f_t({\boldsymbol x}_i)$  we can specify the objective function as:

$$\text{obj}^{[t]} = \sum_{i=1}^n L(y_i, \hat{y}_{i}^{[t-1]} + f_t({\boldsymbol x}_i)) + \sum_{i=1}^t \Omega(f_i)$$

The first term is the squared defference between the actual and predicted value for all $n$ observations when we consider the fact that it sums from 1 to $n$. The second part measures the complexity of all the $t$ trees. To optimise this objective function, in using gradient descent, we need to compute gradients. According to an author of the XGBoost package in R Tong He, the performance can also be improved by considering both the first and second order gradients ($\frac{\partial\text{obj}^{[t]}}{\partial\hat{y}_i^{[t]}}$ and $\frac{\partial^2 \text{obj}^{[t]}}{{\partial\hat{y}_i^{[t]}}^2}$ respectively) as it will allow gradient descent to converge faster:


Since it isn't always possible to compute the first and second order derivatives for every objective function, we can approximate our objective function using a second order [taylor polynomial](https://en.wikipedia.org/wiki/Taylor_series) approximation such that:

$$\text{obj}^{[t]} \approx \sum_{i=1}^n [L(y_i, \hat{y}_{i}^{[t-1]}) + \frac{\partial L(y_i, \hat{y}_{i}^{[t-1]})}{\partial\hat{y}_{i}^{[t-1]}} (\hat{y}_{i}^{[t]} - \hat{y}_{i}^{[t-1]}) + \frac{1}{2}\frac{\partial^2 L(y_i, \hat{y}_{i}^{[t-1]})}{\partial {\hat{y}_i}^{{[t-1]}^2} } (\hat{y}_{i}^{[t]} - \hat{y}_{i}^{[t-1]})^2] + \sum_{i=1}^t \Omega(f_i)$$

Now since $\hat{y}_{i}^{[t]} - \hat{y}_{i}^{[t-1]} = f_t({\boldsymbol x}_i)$ this quantity becomes:

$$\text{obj}^{[t]} \approx \sum_{i=1}^n [L(y_i, \hat{y}_{i}^{[t-1]}) + \frac{\partial L(y_i, \hat{y}_{i}^{[t-1]})}{\partial\hat{y}_{i}^{[t-1]}} f_t({\boldsymbol x}_i) + \frac{1}{2}\frac{\partial^2 L(y_i, \hat{y}_{i}^{[t-1]})}{\partial {\hat{y}_i}^{{[t-1]}^2}} (f_t({\boldsymbol x}_i))^2] + \sum_{i=1}^t \Omega(f_i)$$

Letting $g_i =\frac{\partial L(y_i, \hat{y}_{i}^{[t-1]})}{\partial\hat{y}_{i}^{[t-1]}}$ (the first order gradient statistic of the loss function) and $h_i = \frac{\partial^2 L(y_i, \hat{y}_{i}^{[t-1]})}{\partial {\hat{y}_i}^{{[t-1]}^2}}$ (the second order gradient statistic of the loss function), we get:

$$\text{obj}^{[t]} \approx \sum_{i=1}^n [L(y_i, \hat{y}_{i}^{[t-1]}) + g_i f_t({\boldsymbol x}_i) + \frac{1}{2}h_i f_t({\boldsymbol x}_i))^2] + \sum_{i=1}^t \Omega(f_i)$$

If we remove the constant terms and remove all the complexity measures (regularisation terms) for the previous ($t-1$) trees (which are also constants), we get:

$$\text{obj}^{[t]} \approx \sum_{i=1}^n [g_i f_t({\boldsymbol x}_i) + \frac{1}{2}h_i f_t({\boldsymbol x}_i))^2] + \Omega(f_t)$$

which now (importantly) represents the objective at the $t^{th}$ step. The goal is to now find an $f_t$ to optimise our objective function. 

--- 


The question now becomes in our $t^{th}$ iteration, how can we find a tree that improves the prediction along the gradient? Since our function is a tree, we can't just find a formula to calculate it. Two other questions arise, such as:

1. How can we find a good tree structure?
2. How should we assign prediction scores? IE when data points reach a leaf they are assigned a weight which is the prediction, but how should these weights be calculated?

It is these two questions that we want to solve using gradient descent.

Assuming we have found a good tree structure (ie answered the first question), we can mathematically define a tree as

$$f_t(x) = w_{q(x)}$$

where according to the authors of XGBoost, $w$ is the vector of scores on leaves, $q(x)$ is a "directing" function which assigns every data point to the $q(x)^{th}$ leaf in the current tree, and $T$ is the number of leaves. In other words, we:

1. Assign a data point $x$ to a leaf by $q$
2. Assign the corresponding score $w_{q(x)}$ on the $q(x)^{th}$ leaf to the data point (row of data). IE all observations that fall into a particular leaf will be assigned a score

---

We now define a quantity known as:

$$I_j = \{i|q(x) = j\}$$

which represents the set of indices all data points that fell in the $j^{th}$ leaf

Recall that the objective function was given as 

$$\text{obj}^{[t]} \approx \sum_{i=1}^n [g_i f_t({\boldsymbol x}_i) + \frac{1}{2}h_i f_t({\boldsymbol x}_i))^2] + \gamma{T} + \frac{1}{2}\lambda \sum_{j=1}^T {w}_j^2$$

if we substitute in $\Omega = \gamma{T} + \frac{1}{2}\lambda \sum_{j=1}^T {w}_j^2.$

If we now utilise the definition of what $I_j$ is, we obtain another way of expressing $\text{obj}^{[t]}$ such that:

$$\text{obj}^{[t]} \approx \sum_{j=1}^T \big[(\sum_{i \in I_j} g_i)w_j + \frac{1}{2}(\sum_{i \in I_j} h_i + \lambda)w_{j}^2 \big] + \gamma{T}$$

if we are to factorise the $\frac{1}{2}$, recognise that $f_t({\boldsymbol x}_i) = w_j$ and sum across all $T$ leaves, finding the observations $i \in I_j$ that fell in the $j^{th}$ leaf. The indexes of the summation change since we know that data points at the same leaf get an identical score. Moving forward, it might be easier to let $G_j = \sum_{i \in I_j} g_i$ and $H_i = \sum_{i \in I_j}h_i$, hence:

$$\text{obj}^{[t]} \approx \sum_{j=1}^T \big[G_jw_j + \frac{1}{2}(H_j + \lambda)w_{j}^2 \big] + \gamma{T}$$

---

Now, we want to find weights (leaf scores) $w_j$ to optimise our objective function at iteration $t$. This problem has just a squared $w_j$ term and a linear $w_j$ term and is thus quadratic. We have $T$ of these quadratic functions to sum over. Ultimatley, for each quadratic function, we get 

$$G_jw_j + \frac{1}{2}(H_j + \lambda)w_{j}^2$$

Differentiating with respect to $w_j$, you get:

$$\frac{\partial}{\partial w_j} G_jw_j + \frac{1}{2}(H_j + \lambda)w_{j}^2 = G_j + (H_j + \lambda)w_j$$

Setting this derivative to 0 to find the stationary point (which is a minimum in this case) and solving for $w_j$, we get

$$w_j^{*} = -\frac{G_j}{(H_j + \lambda)} = -\frac{\sum_{i \in I_j} g_i}{\sum_{i \in I_j} (h_i + \lambda)}$$

We now see that the leaf score $w_j = -\frac{\sum_{i \in I_j} g_i}{\sum_{i \in I_j} (h_i + \lambda)}$ is comprised of both the first and second order gradients of the loss function $g$ asnd $h$ as well as the $\lambda$ term which controls regularisation.

If we substitute this term into our $\text{obj}^{[t]}$, we get:

$$\text{obj}^{[t]} \approx -\frac{1}{2} \sum_{j=1}^T \frac{G_j^2}{H_j + \lambda} + \gamma{T} = -\frac{1}{2} \sum_{j=1}^T \frac{(\sum_{i \in I_j} g_i)^2}{\sum_{i \in I_j} h_i + \lambda} + \gamma{T}$$

**Note: In the [original XGBoost paper](https://arxiv.org/pdf/1603.02754.pdf) they mention that "this score is like the impurity
score for evaluating decision trees, except that it is derived for a wider range of objective functions" and "also takes the model complexity into account." They also mention that "the smaller the score is, the better the structure is"**

So finding $w_j$ answers the question as to how we should assign leaf values for an individual tree. Going back to the first question of how to build the tree structure in the first place.

* To build the tree we start with a depth of 0 (just a root node)
* For each leaf node of the tree, try to add a split. The change of objective after adding the split is given as 

$$Gain = \frac{1}{2}\big(\frac{G_{L}^2}{H_L + \lambda} + \frac{G_{R}^2}{H_R + \lambda} + \frac{(G_L + G_R)^2}{H_L + H_R + \lambda}\big) - \gamma = \frac{1}{2}\big(\frac{(\sum_{i \in I_L} g_i)^2}{H_L + \lambda} + \frac{(\sum_{i \in I_R} g_i)^2}{H_R + \lambda} + \frac{(\sum_{i \in I_L \cup I_R} g_i)^2}{H_L + H_R + \lambda}\big) - \gamma$$

whereby the first term represents the score of a the left child, the second term is the score of the right child, the third term is the score if we do not split and the fourth term $\gamma$ is a complexity cost due to introducing a new leaf. If 'Gain' is negative (i.e. when the training loss reduction is less than the regularisation), then we do not add any more nodes to the tree, and thus helps create shorter trees that are less susceptable to overfitting since we acknowledge that there is in fact a trade-off between predictiveness and simplicity. To actually determine the split, we find a feature $X_j$ and cutoff point $X_j = c$ that maximises gain.

It is common to incorporate post-pruning, in which we grow the tree to its max depth, and recursively prune all the leaf splits depending on whether the 'Gain' was positive or negative. 

One very powerful component of XGBoost is that it can internally handle missing values. For each node, we guide all the data points with a missing value

* To the left sub-node, and calculate the maximum gain
* To the right sub-node, and calculate the maximum gain
* Choose the direction with a larger gain

Though while XGBoost can handle missing values internally, many recommend using other methods of missing value handling such as KNN Imputation

In [ ]:
# Python files
import data_prep
import helper_funcs

import joblib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import xgboost as xgb

# Personal display settings
#===========================

# Suppress scientific notation
np.set_printoptions(suppress=True)

# Get dataset values showing only 2dp
pd.options.display.float_format = '{:.2f}'.format
pd.set_option('display.max_colwidth', None)

plt.style.use('seaborn-whitegrid') 
%matplotlib inline

%load_ext autoreload
%autoreload 2

In [ ]:
train = pd.read_csv('../datasets/train_updated.csv')
test = pd.read_csv('../datasets/test_updated.csv')

# Split data
to_drop = ['Country', 'HDI', 'Life_exp']

X_train = train.drop(to_drop, axis='columns')
X_test = test.drop(to_drop, axis='columns')

y_train = train['Life_exp']
y_test = test['Life_exp']

In [ ]:
xgboost_pipeline = data_prep.create_pipeline(xgb.XGBRegressor(seed=1))

## Hyperparameter Tuning

In [ ]:
param_grid = {
    'imputation__n_neighbors': np.arange(3, 21, 2), 
    'imputation__weights': ['uniform', 'distance'], 
    'model__num_boosting_rounds': np.arange(100, 500, 5),  # n_estimators
    'model__base_score': np.array([0.5, np.mean(y_train)]),  # initial prediction
    'model__learning_rate': np.linspace(0.01, 3, 10),
    'model__max_depth': np.arange(4, 9),
    'model__gamma': np.linspace(0, 100, 10),
    'model__reg_lambda': np.linspace(0, 100, 10),
    'model__eval_metric': ['rmse', 'mae']
}

final_model, best_params = data_prep.randomised_search_wrapper(X_train,
                                                               y_train,
                                                               xgboost_pipeline, 
                                                               param_grid, 
                                                               cv=10,
                                                               n_iter=25)

## Evaluation Metrics

In [ ]:
r2, mse, rmse, mae = helper_funcs.display_regression_metrics(y_test, final_model.predict(X_test))

## Save Model 

In [ ]:
joblib.dump(final_model, './saved_models/XGBoost.joblib')